In [1]:
import pandas as pd
import sklearn.feature_extraction as fe
import sklearn.metrics as m

#
import pickle
import json
import os

Notebook-only

In [2]:
#
original_directory = os.getcwd()
os.chdir("../../data/raw")
os.listdir()

['20News_coarse.pkl',
 'nyt_seedwords.json',
 '20News_seedwords.json',
 'nyt_coarse.pkl']

In [3]:
# read data

#
with open("nyt_coarse.pkl", "rb") as fp:
    data_nyt = pickle.load(fp)
with open("20News_coarse.pkl", "rb") as fp:
    data_20news = pickle.load(fp)
    
with open("nyt_seedwords.json") as fp:
    seed_nyt = json.load(fp)
with open("20News_seedwords.json") as fp:
    seed_20news = json.load(fp)

In [4]:
#
data_nyt.head()
# data_nyt.iloc[0]["sentence"]

,sentence,label
0,"nasa, in preparation for a spacewalk on saturd...",science
1,if professional pride and strong defiance can ...,sports
2,"admittedly, the language is reconstructed and ...",arts
3,"palermo, sicily — roberta vinci beat top-seede...",sports
4,the argentine soccer club san lorenzo complete...,sports


In [5]:
#
data_20news.head() #newline characters visible

,sentence,label
0,from: (where's my thing)\nsubject: what car i...,rec
1,from: (guy kuo)\nsubject: si clock poll - fin...,comp
2,from: (thomas e willis)\nsubject: pb question...,comp
3,from: jgreen@amber (joe green)\nsubject: re: w...,comp
4,from: (jonathan mcdowell)\nsubject: re: shutt...,sci


In [6]:
#
seed_nyt #same format as 20News seed words

{'arts': ['music', 'orchestra', 'album', 'opera', 'ballet'],
 'business': ['companies', 'euro', 'economy', 'batteries', 'sales'],
 'science': ['space', 'researchers', 'scientists', 'research', 'science'],
 'sports': ['yankees', 'cup', 'league', 'basketball', 'golf'],
 'politics': ['republicans', 'senator', 'senate', 'democrats', 'election']}

In [40]:
seed_20news

{'alt': ['atheism', 'atheists', 'religion', 'objective'],
 'comp': ['graphics', 'windows', 'scsi', 'mac'],
 'misc': ['sale', 'offer', 'shipping', 'forsale'],
 'rec': ['car', 'bike', 'game', 'team'],
 'sci': ['encryption', 'circuit', 'candida', 'space'],
 'talk': ['turkish', 'gun', 'jews', 'armenian'],
 'soc': ['church', 'jesus', 'christ', 'christians']}

# TF-IDF

## NYT

IR-TF-IDF treats the seed word set for each class as a query. The relevance of a document to a label is computed by aggregated TF-IDF values of its respective seed words. The label with the highest relevance is assigned to each document.

In [7]:
# genres_nyt = {}
# for genre,seed_words in seed_nyt.items():
#     for seed_word in seed_words:
#         genres_nyt[seed_word] = genre
        
# genres_nyt

In [8]:
# ir-tf-idf replication

# intialize vectorizer
# tfidf_model = fe.text.TfidfVectorizer(input="content", stop_words = {'english'})
# tfidf_vector = tfidf_model.fit_transform(data_nyt["sentence"])

In [9]:
#get vector with
# filtered = list(filter(lambda item: item[1] in genres_nyt, zip(tfidf_vector[0].toarray()[0].tolist(),tfidf_model.get_feature_names_out())))

In [10]:
# list(filtered)
# label_test = max(filtered, key=lambda item: item[0])
# label_test[1]

In [11]:
# nyt_tfidf = dict(zip(tfidf_model.get_feature_names_out()[i],tfidf_vector[].toarray()[0].tolist()[i]) for i in range(
#     len(tfidf_model.get_feature_names_out())))
# nyt_tfidf

In [12]:
# tfidf_model.get_feature_names_out()

In [41]:
def ir_tfidf(data, seeds):
    '''
    Return a DataFrame of the input with predicted labels using IR-TF-IDF
    
    :param: data: a DataFrame with a "sentence" attribute
    :param: seeds: a dictionary of labels (keys) with a list of seed words (values)
    '''
    #reverse seeds dictionary
    genres = {}
    for genre,seed_words in seeds.items():
        for seed_word in seed_words:
            genres[seed_word] = genre
            
    data_ind = data.reset_index()
            
    model = fe.text.TfidfVectorizer(input="content", stop_words = {'english'})
    vector = model.fit_transform(data["sentence"])
    features = model.get_feature_names() #ADD _OUT FOR RECENT SKLEARN VERSION
    
    def predict(ind):
        filtered = filter(lambda item: item[1] in genres, zip(vector[ind].toarray()[0].tolist(),features))
        prediction = max(filtered, key=lambda item: item[0])
        return genres[prediction[1]]
    
    df = data_ind.assign(prediction = data_ind["index"].apply(predict))
    return df

In [18]:
tfidf_nyt = ir_tfidf(data_nyt,seed_nyt)

In [19]:
tfidf_nyt

,index,sentence,label,prediction
0,0,"nasa, in preparation for a spacewalk on saturd...",science,science
1,1,if professional pride and strong defiance can ...,sports,sports
2,2,"admittedly, the language is reconstructed and ...",arts,arts
3,3,"palermo, sicily — roberta vinci beat top-seede...",sports,arts
4,4,the argentine soccer club san lorenzo complete...,sports,sports
...,...,...,...,...
11522,11522,"melbourne, australia — after the tears and the...",sports,arts
11523,11523,"perth, australia — wimbledon semifinalist jerz...",sports,sports
11524,11524,while the nets are hoisting no. 5 to the barcl...,sports,sports
11525,11525,sydney — tennis australia chief executive stev...,sports,arts


In [33]:
def score_F1(labels, predictions, averaging="micro"):
    '''
    Returns the micro F1 score given a list of labels and a list of predictions
    
    :param: labels: a list of labels
    :param: predictions: a list of predictions
    :param: averaging: type of averaging method to use
    '''
    score = m.f1_score(labels, predictions, average=averaging)
    return score

In [34]:
micro_nyt_tfidf = score_F1(tfidf_nyt["label"], tfidf_nyt["prediction"], "micro")
macro_nyt_tfidf = score_F1(tfidf_nyt["label"], tfidf_nyt["prediction"], "macro")

In [35]:
micro_nyt_tfidf

0.6361585841936324

In [36]:
macro_nyt_tfidf

0.49765851868243194

In [42]:
tfidf_20News = ir_tfidf(data_20news,seed_20news)

In [43]:
tfidf_20News

,index,sentence,label,prediction
0,0,from: (where's my thing)\nsubject: what car i...,rec,rec
1,1,from: (guy kuo)\nsubject: si clock poll - fin...,comp,talk
2,2,from: (thomas e willis)\nsubject: pb question...,comp,comp
3,3,from: jgreen@amber (joe green)\nsubject: re: w...,comp,comp
4,4,from: (jonathan mcdowell)\nsubject: re: shutt...,sci,talk
...,...,...,...,...
18254,18254,from: (stupendous man)\nsubject: re: temperat...,sci,sci
18255,18255,from: (jim smyton)\nsubject: re: monitors - s...,comp,talk
18256,18256,from: \nsubject: re: game length (was re: brav...,rec,rec
18257,18257,from: \nsubject: intel chmos 8086/8088 design...,misc,misc
